In [ ]:
#| default_exp  Features.pitch_control

In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

In [ ]:
#| export

import collections
import math
import os
import re
from pathlib import Path

import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.figure_factory as ff
from omnisync.visualization.pitch import Pitch,plot_pitch

from fastcore.foundation import L

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [ ]:
data_path = Path("../data")

tracking_event_mapping = pd.read_csv(
    data_path / "tracking_event_mapping.csv", low_memory=False
)
tracking_df = pd.read_csv(
    data_path / "tracking_vel_df.csv", low_memory=False
).set_index("frameId")

## pick the frame-id associated with a randomly picked event
frame_id = 135036

## select the frame
frame = tracking_df.loc[frame_id]
lineup = pd.read_csv(data_path / "lineup.csv")

In [ ]:
class inputs_:
    def __init__(self, frame, events_to_frame, lineup):
        self.frame, self.lineup, self.events_to_frame = frame, lineup, events_to_frame
        self.gk_ids = lineup[lineup.position == "GK"].playerId.tolist()
        self.frame_id = frame.name
        self.possession_team_id = events_to_frame.loc[
            events_to_frame.frameId == self.frame_id, "teamId"
        ].values[0]
        self.possession_side = self.lineup.loc[
            self.lineup.teamId == self.possession_team_id, "side"
        ].values[0]
        self.opponent_side = "away" if bool(re.match("home",self.possession_side)) else "home"
        self.ball_carrier_id = events_to_frame.loc[
            events_to_frame.frameId == self.frame_id, "playerId"
        ].values[0]
        self.ball = [self.frame["ball_x"],self.frame["ball_y"]]
    def _prepare_inputs(self):
        _player_id_cols = self.frame.filter(like="_player_id").index.tolist()
        linputs = L(
                {
                    "playerId": self.frame[pid_col],
                    "playerStr": pid_col.removesuffix("_player_id"),
                    "jerseyNumber": int(
                        pid_col.removesuffix("_player_id")
                        .removeprefix("away_player_")
                        .removeprefix("home_player_")
                    ),
                    "side": "home" if bool(re.match("home", pid_col)) else "away",
                    "x": self.frame[pid_col.removesuffix("_player_id") + "_x"],
                    "y": self.frame[pid_col.removesuffix("_player_id") + "_y"],
                    "vx": self.frame[pid_col.removesuffix("_player_id") + "_vx"],
                    "vy": self.frame[pid_col.removesuffix("_player_id") + "_vy"],
                }
                for pid_col in _player_id_cols
            )
        
        self.inputs = pd.DataFrame(linputs).dropna(axis=0)
        
        self.att = self.inputs[self.inputs["side"] == self.possession_side] 
        self.defe = self.inputs[self.inputs["side"] == self.opponent_side]
        
        self.att_gk_id = self.gk_ids[0] if self.gk_ids[0] in self.att.playerId else self.gk_ids[1]
        self.def_gk_id = self.gk_ids[0] if self.gk_ids[0] in self.defe.playerId else self.gk_ids[1]
        
        #self.gk_def_idx = pd.Index(self.defe["playerId"].values.tolist()).get_loc(self.def_gk_id)
        
        self.att_tracking= self.att[["x","y","vx","vy"]].to_numpy()
        self.def_tracking= self.defe[["x","y","vx","vy"]].to_numpy()

In [ ]:
def default_model_params(time_to_control_veto=3):
    # key parameters for the model, as described in Spearman 2018
    params = {}
    # model parameters
    params['max_player_accel'] = 7. # maximum player acceleration m/s/s, not used in this implementation
    params['max_player_speed'] = 5. # maximum player speed m/s
    params['reaction_time'] = 0.7 # seconds, time taken for player to react and change trajectory. Roughly determined as vmax/amax
    params['tti_sigma'] = 0.45 # Standard deviation of sigmoid function in Spearman 2018 ('s') that determines uncertainty in player arrival time
    params['kappa_def'] =  1.72 # kappa parameter in Spearman 2018 (=1.72 in the paper) that gives the advantage defending players to control ball, I have set to 1 so that home & away players have same ball control probability
    params['lambda_att'] = 4.3 # ball control parameter for attacking team
    params['lambda_def'] = 4.3 * params['kappa_def'] # ball control parameter for defending team
    params['lambda_gk'] = params['lambda_def']*3.0 # make goal keepers must quicker to control ball (because they can catch it)
    params['average_ball_speed'] = 15. # average ball travel speed in m/s
    # numerical parameters for model evaluation
    params['int_dt'] = 0.04 # integration timestep (dt)
    params['max_int_time'] = 10 # upper limit on integral time
    params['model_converge_tol'] = 0.01 # assume convergence when PPCF>0.99 at a given location.
    # The following are 'short-cut' parameters. We do not need to calculated PPCF explicitly when a player has a sufficient head start. 
    # A sufficient head start is when the a player arrives at the target location at least 'time_to_control' seconds before the next player
    params['time_to_control_att'] = time_to_control_veto*np.log(10) * (np.sqrt(3)*params['tti_sigma']/np.pi + 1/params['lambda_att'])
    params['time_to_control_def'] = time_to_control_veto*np.log(10) * (np.sqrt(3)*params['tti_sigma']/np.pi + 1/params['lambda_def'])
    return params

In [ ]:
p = inputs_(frame,tracking_event_mapping,lineup)
p._prepare_inputs()
df = p.inputs

params = default_model_params()

In [ ]:
class player(object):
    # player object holds position, velocity, time-to-intercept and pitch control contributions for each player
    def __init__(self,pid,team_name,pos,vel,GKid,params):
        self.id = pid
        self.teamname = team_name
        self.is_gk = self.id == GKid
        self.vmax = params['max_player_speed'] # player max speed in m/s. Could be individualised
        self.reaction_time = params['reaction_time'] # player reaction time in 's'. Could be individualised
        self.tti_sigma = params['tti_sigma'] # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_att = params['lambda_att'] # standard deviation of sigmoid function (see Eq 4 in Spearman, 2018)
        self.lambda_def = params['lambda_gk'] if self.is_gk else params['lambda_def'] # factor of 3 ensures that anything near the GK is likely to be claimed by the GK
        self.position = np.array(pos)
        self.velocity = np.array(vel)
        self.PPCF = 0
    
    def simple_time_to_intercept(self, r_final):
        self.PPCF = 0. # initialise this for later
        # Time to intercept assumes that the player continues moving at current velocity for 'reaction_time' seconds
        # and then runs at full speed to the target position.
        r_reaction = self.position + self.velocity*self.reaction_time
        self.time_to_intercept = self.reaction_time + np.linalg.norm(r_final-r_reaction)/self.vmax
        return self.time_to_intercept

    def probability_intercept_ball(self,T):
        # probability of a player arriving at target location at time 'T' given their expected time_to_intercept (time of arrival), as described in Spearman 2018
        f = 1/(1. + np.exp( -np.pi/np.sqrt(3.0)/self.tti_sigma * (T-self.time_to_intercept) ) )
        return f

In [ ]:
def initialise_players(team,team_name,params,gkid):
    player_ids = team["playerId"].tolist()
    pos_x = team["x"].tolist()
    pos_y = team["y"].tolist()
    v_x = team["vx"].tolist()
    v_y = team["vy"].tolist()
    # create a player object for player_id 'p'
    team_players = [
        player(
            pid,
            team_name,
            [x,y],
            [vx,vy],
            gkid,
            params
        )
        for pid,x,y,vx,vy in zip(
            player_ids,
            pos_x,
            pos_y,
            v_x,
            v_y)
    ]
    return team_players

In [ ]:
def calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params):
    
    # calculate ball travel time from start position to end position.
    if ball_start_pos is None or any(np.isnan(ball_start_pos)): # assume that ball is already at location
        ball_travel_time = 0.0 
    else:
        # ball travel time is distance to target position from current ball position divided assumed average ball speed
        ball_travel_time = np.linalg.norm( target_position - ball_start_pos )/params['average_ball_speed']
    
    # first get arrival time of 'nearest' attacking player (nearest also dependent on current velocity)
    tau_min_att = np.nanmin( [p.simple_time_to_intercept(target_position) for p in attacking_players] )
    tau_min_def = np.nanmin( [p.simple_time_to_intercept(target_position ) for p in defending_players] )
    
    # check whether we actually need to solve equation 3
    if tau_min_att-max(ball_travel_time,tau_min_def) >= params['time_to_control_def']:
        # if defending team can arrive significantly before attacking team, no need to solve pitch control model
        return 0., 1.
    elif tau_min_def-max(ball_travel_time,tau_min_att) >= params['time_to_control_att']:
        # if attacking team can arrive significantly before defending team, no need to solve pitch control model
        return 1., 0.
    else: 
        # solve pitch control model by integrating equation 3 in Spearman et al.
        # first remove any player that is far (in time) from the target location
        attacking_players = [p for p in attacking_players if p.time_to_intercept-tau_min_att < params['time_to_control_att'] ]
        defending_players = [p for p in defending_players if p.time_to_intercept-tau_min_def < params['time_to_control_def'] ]
        # set up integration arrays
        dT_array = np.arange(ball_travel_time-params['int_dt'],ball_travel_time+params['max_int_time'],params['int_dt']) 
        PPCFatt = np.zeros_like( dT_array )
        PPCFdef = np.zeros_like( dT_array )
        # integration equation 3 of Spearman 2018 until convergence or tolerance limit hit (see 'params')
        ptot = 0.0
        i = 1
        while 1-ptot>params['model_converge_tol'] and i<dT_array.size: 
            T = dT_array[i]
            for player in attacking_players:
                # calculate ball control probablity for 'player' in time interval T+dt
                dPPCFdT = (1-PPCFatt[i-1]-PPCFdef[i-1])*player.probability_intercept_ball( T ) * player.lambda_att
                # make sure it's greater than zero
                assert dPPCFdT>=0, 'Invalid attacking player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT*params['int_dt'] # total contribution from individual player
                PPCFatt[i] += player.PPCF # add to sum over players in the attacking team (remembering array element is zero at the start of each integration iteration)
            for player in defending_players:
                # calculate ball control probablity for 'player' in time interval T+dt
                dPPCFdT = (1-PPCFatt[i-1]-PPCFdef[i-1])*player.probability_intercept_ball( T ) * player.lambda_def
                # make sure it's greater than zero
                assert dPPCFdT>=0, 'Invalid defending player probability (calculate_pitch_control_at_target)'
                player.PPCF += dPPCFdT*params['int_dt'] # total contribution from individual player
                PPCFdef[i] += player.PPCF # add to sum over players in the defending team
            ptot = PPCFdef[i]+PPCFatt[i] # total pitch control probability 
            i += 1
        if i>=dT_array.size:
            print("Integration failed to converge: %1.3f" % (ptot) )
        return PPCFatt[i-1], PPCFdef[i-1]

In [ ]:
def generate_pitch_control_for_event(frame, params, field_dimen = (105.,68.,), n_grid_cells_x = 105):
  
    tracking_home = frame.inputs[frame.inputs["side"] == "home"] 
    tracking_away = frame.inputs[frame.inputs["side"] == "away"]
    
    # get the details of the event (frame, team in possession, ball_start_position)
    pass_frame = frame.inputs
    pass_team = frame.possession_side
    ball_start_pos = np.array(frame.ball)
    # break the pitch down into a grid
    n_grid_cells_y = int(n_grid_cells_x*field_dimen[1]/field_dimen[0])
    dx = field_dimen[0]/n_grid_cells_x
    dy = field_dimen[1]/n_grid_cells_y
    xgrid = np.arange(n_grid_cells_x)
    ygrid = np.arange(n_grid_cells_y)
    # initialise pitch control grids for attacking and defending teams 
    PPCFa = np.zeros( shape = (len(ygrid), len(xgrid)) )
    PPCFd = np.zeros( shape = (len(ygrid), len(xgrid)) )
    # initialise player positions and velocities for pitch control calc (so that we're not repeating this at each grid cell position)
    if pass_team == 'home':
        attacking_players = initialise_players(tracking_home ,'home',params,frame.att_gk_id)
        defending_players = initialise_players(tracking_away,'away',params,frame.def_gk_id)
    elif pass_team == 'away':
        defending_players = initialise_players(tracking_home,'home',params,frame.def_gk_id)
        attacking_players = initialise_players(tracking_away,'away',params,frame.att_gk_id)
    else:
        assert False, "Team in possession must be either home or away"
        
    # find any attacking players that are offside and remove them from the pitch control calculation
    
    # calculate pitch pitch control model at each location on the pitch
    for i in range( len(ygrid) ):
        for j in range( len(xgrid) ):
            target_position = np.array( [xgrid[j], ygrid[i]] )
            PPCFa[i,j],PPCFd[i,j] = calculate_pitch_control_at_target(target_position, attacking_players, defending_players, ball_start_pos, params)
    # check probabilitiy sums within convergence
    checksum = np.sum( PPCFa + PPCFd ) / float(n_grid_cells_y*n_grid_cells_x ) 
    assert 1-checksum < params['model_converge_tol'], "Checksum failed: %1.3f" % (1-checksum)
    return PPCFa,xgrid,ygrid

In [ ]:
PPCFa,xgrid,ygrid = generate_pitch_control_for_event(p,params, field_dimen = (105.,68.,), n_grid_cells_x = 105)

In [ ]:
PPCFa[0, 0]

0.0020650507050388824